In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt


In [4]:
def extract_answer(text):
    pattern = r"(True|False|Don't know)"
    matches = re.findall(pattern, text)
    return matches


In [63]:
RESPONSE_PATH = "../../../data/analysis_results/Llama_2_13b_chat_hf_prompt_based_binary_response.csv"

# RESPONSE_PATH = "../../../data/analysis_results/Llama_2_13b_chat_hf_node_retrieval_rag_based_binary_response.csv"


In [64]:
response_df = pd.read_csv(RESPONSE_PATH)

response_df.loc[:, 'extracted_answer'] = response_df['llm_answer'].apply(extract_answer)


In [65]:
response_df.loc[:, "answer_count"] = response_df.extracted_answer.apply(lambda x:len(x))

response_df_multiple_answers = response_df[response_df.answer_count > 1]
response_df_single_answer = response_df.drop(response_df_multiple_answers.index)
response_df_single_answer.drop("answer_count", axis=1, inplace=True)


response_df_multiple_answers_ = []
for index, row in response_df_multiple_answers.iterrows():
    if row["extracted_answer"][0] == row["extracted_answer"][1]:
        response_df_multiple_answers_.append((row["question"], row["label"], row["llm_answer"], row["extracted_answer"][0]))
    else:
        response_df_multiple_answers_.append((row["question"], row["label"], row["llm_answer"], None))

response_df_multiple_answers_ = pd.DataFrame(response_df_multiple_answers_, columns=["question", "label", "llm_answer", "extracted_answer"])

response_df_final = pd.concat([response_df_single_answer, response_df_multiple_answers_], ignore_index=True)
response_df_final = response_df_final.explode("extracted_answer")



In [66]:
incorrect_answers_because_of_na = response_df_final[response_df_final.extracted_answer.isna()]


In [67]:
response_transform = {
    "True" : True,
    "False" : False
}

response_df_final.extracted_answer = response_df_final.extracted_answer.apply(lambda x:response_transform[x])



In [68]:
correct_response = response_df_final[response_df_final.label == response_df_final.extracted_answer]
incorrect_response = response_df_final[response_df_final.label != response_df_final.extracted_answer]



In [69]:
total_response = response_df_final.shape[0]
correct_response_count = correct_response.shape[0]
incorrect_response_count = incorrect_response.shape[0] + incorrect_answers_because_of_na.shape[0]

In [70]:
correct_response_ = correct_response_count/total_response
incorrect_response_ = incorrect_response_count/total_response


print("Correct response = ",correct_response_)
print("Incorrect response = ",incorrect_response_)



Correct response =  0.8899835796387521
Incorrect response =  0.11001642036124795


In [71]:
correct_response.llm_answer.values[-1]

'  {\n"answer": "True"\n}\n\nJuvenile polyposis syndrome (JPS) is a rare genetic disorder characterized by the development of multiple juvenile polyps in the gastrointestinal tract. One of the genes associated with JPS is BMPR1A. Mutations in the BMPR1A gene have been identified as a cause of JPS, and individuals with these mutations have an increased risk of developing colorectal cancer at a young age. Therefore, the answer is "True" that JPS associates with the BMPR1A gene.'

In [72]:
correct_response.question.values[0]

'enhanced S-cone syndrome is not a vitreoretinal degeneration'

In [73]:
correct_response.shape

(542, 4)

In [74]:
correct_response

,question,label,llm_answer,extracted_answer
0,enhanced S-cone syndrome is not a vitreoretina...,False,"{\n""answer"": ""False""\n}\n\nEnhanced S-cone s...",False
2,KLEEFSTRA SYNDROME 1 is not associated with Ge...,False,"{\n ""answer"": ""False""\n}\n\nKleefstra syndr...",False
3,STARGARDT DISEASE 1 (disorder) is not associat...,False,"{\n ""answer"": ""False""\n}\n\nStargardt disea...",False
4,Juvenile polyposis syndrome associates Gene SMAD4,True,"{\n""answer"": ""True""\n}\n\nJuvenile polyposis...",True
5,Glycogen storage disease type II associates Ge...,True,"{\n""answer"": ""True""\n}",True
...,...,...,...,...
604,Gerstmann-Straussler-Scheinker Disease associa...,True,"{\n""answer"": ""True""\n}\n\nGerstmann-Straussl...",True
605,Congenital Generalized Lipodystrophy Type 2 as...,True,"{\n""answer"": ""True""\n}\n\nCongenital General...",True
606,Hidrotic Ectodermal Dysplasia associates Gene ...,True,"{\n""answer"": ""True""\n}\n\nHidrotic Ectoderma...",True
607,"TIBIAL MUSCULAR DYSTROPHY, TARDIVE associates ...",True,"{\n""answer"": ""True""\n}\n\nTibial muscular dy...",True
